Zuerst werden alle notwendigen Pakete und Module importiert.
Dabei wird neben den Modulen der Standardbibliothek (`random`, `sys` und `time`) und des `defaultdict` aus dem `collections` Modul noch das Paket *Python-Chess* (`chess`) benötigt, welches bereits installiert wurde.
Zudem werden für die Anzeige einige Funktionen aus IPython benötigt, die durch die Installation und Verwendung von Jupyter Notebooks bereitgestellt werden.

In [ ]:
import random
import sys
import time

import chess
import chess.engine
import chess.polyglot
import chess.svg

from collections import defaultdict

from IPython.display import display, HTML, clear_output

Als nächstes wird eine globale Konstante `TIMEOUT_SECONDS` definiert.
Diese wird verwendet, um die Zeit des Schachcomputers zur Berechnung seines nächsten Zuges festzulegen.

In [ ]:
TIMEOUT_SECONDS: int = 30

Im folgenden Schritt werden einige globale Variablen definiert.
Dieser Schritt ist notwendig, um die Funktionsköpfe möglichst nah am Pseudocode aus dem Theorieteil und der Literatur zu halten.

- Die Variable `best_move` enthält den besten Zug für die aktuelle Iteration der `iterative_deepening`-Funktion.
- Demgegenüber speichert die Variable `global_best_move` den besten Zug aus allen Iterationen der iterativen Tiefensuche und somit den Zug, der vom Schachcomputer am Ende seines Zuges ausgeführt wird.
- `current_depth` speichert die aktuelle Tiefe für das Iterative Deepening.
- `is_timeout` ist genau dann wahr, wenn die Zeit des Schachcomputers abgelaufen ist.
- Bei `move_scores` handelt es sich um ein Dictionary, dass den Score für jeden bereits besuchten Zug im Iterative Deepening Verfahren speichert.
Dabei handelt es sich speziell um ein verschachteltes Dictionary.
Die Schlüssel auf der obersten Ebene bilden die Stellungen in FEN Notation.
Der dazugehörige Wert ist wiederum ein Dictionary mit den Zügen als Schlüssel und dem dazugehörigen Score als Wert.
- `neighbor_boards` ist ein Dictionary, dass alle Stellungen enthält, die in einem Halbzug von der aktuellen Stellung aus erreicht werden können und die evaluiert wurden.
Es dient dazu, das Dictionary `move_scores` zu beschneiden.
- In der Variablen `start_time` wird die Startzeit des Zuges des Schachcomputers abgelegt.

In [ ]:
best_move = None
current_depth = 0
global_best_move = None
is_timeout: bool = False
move_scores = defaultdict(dict)
neighbor_boards = defaultdict(list)
start_time: float = 0.0

Nachdem alle notwendigen globalen Definitionen getätigt worden sind, wird nun die verwendete Bewertungsheuristik umgesetzt.
Da sich die Bewertungsfunktion in zwei Teile aufteilt, werden diese getrennt implementiert und am Ende zusammengefügt.
Zuerst erfolgt die Umsetzung der Figurenwerte.
Hierfür wird ein Dictionary `piece_values` angelegt, dass jeder Figur ihren entsprechenden Wert zuweist.

In [ ]:
piece_values = {
    chess.BISHOP: 330,
    chess.KING: 20_000,
    chess.KNIGHT: 320,
    chess.PAWN: 100,
    chess.QUEEN: 900,
    chess.ROOK: 500,
}

Die Funktion `get_piece_value` gibt den Figurenwert für eine übergebene Figur auf dem Schachbrett zurück.
Handelt es sich bei der Figur um eine der Farbe Schwarz, so wird der negierte Werte zurückgegeben, da der Schwarze Spieler der minimierende Spieler ist.

In [ ]:
def get_piece_value(piece: chess.Piece) -> int:
    factor = -1 if piece.color == chess.BLACK else 1
    return factor * piece_values.get(piece.piece_type)

Als nächstes werden die figurenspezifischen Positionstabellen implementiert.
Dafür wird ein Dictionary `piece_squared_tables` angelegt, welches für jede Figur die entsprechende Positionstabelle speichert.
Die Tabellen werden dabei als Tupel von Tupel gespeichert, da die Veränderung der Werte während des Spiels nicht zulässig ist.
Die Tabellen sind aus dem Theorieteil übernommen worden, weshalb sie aus Sicht des weißen Spielers zu betrachten sind.
Das Feld A1 befindet sich somit unten links, was dem Index `[7][0]` entspricht.

In [ ]:
piece_squared_tables = {
    chess.BISHOP: (
        (-20, -10, -10, -10, -10, -10, -10, -20),
        (-10,   0,   0,   0,   0,   0,   0, -10),
        (-10,   0,   5,  10,  10,   5,   0, -10),
        (-10,   5,   5,  10,  10,   5,   5, -10),
        (-10,   0,  10,  10,  10,  10,   0, -10),
        (-10,  10,  10,  10,  10,  10,  10, -10),
        (-10,   5,   0,   0,   0,   0,   5, -10),
        (-20, -10, -10, -10, -10, -10, -10, -20),
    ),
    chess.KING: (
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-30, -40, -40, -50, -50, -40, -40, -30),
        (-20, -30, -30, -40, -40, -30, -30, -20),
        (-10, -20, -20, -20, -20, -20, -20, -10),
        ( 20,  20,   0,   0,   0,   0,  20,  20),
        ( 20,  30,  10,   0,   0,  10,  30,  20),
    ),
    chess.KNIGHT: (
        (-50, -40, -30, -30, -30, -30, -40, -50),
        (-40, -20,   0,   0,   0,   0, -20, -40),
        (-30,   0,  10,  15,  15,  10,   0, -30),
        (-30,   5,  15,  20,  20,  15,   5, -30),
        (-30,   0,  15,  20,  20,  15,   0, -30),
        (-30,   5,  10,  15,  15,  10,   5, -30),
        (-40, -20,   0,   5,   5,   0, -20, -40),
        (-50, -40, -30, -30, -30, -30, -40, -50),
    ),
    chess.PAWN: (
        (  0,   0,   0,   0,   0,   0,   0,   0),
        ( 50,  50,  50,  50,  50,  50,  50,  50),
        ( 10,  10,  20,  30,  30,  20,  10,  10),
        (  5,   5,  10,  25,  25,  10,   5,   5),
        (  0,   0,   0,  20,  20,   0,   0,   0),
        (  5,  -5, -10,   0,   0, -10,  -5,   5),
        (  5,  10,  10, -20, -20,  10,  10,   5),
        (  0,   0,   0,   0,   0,   0,   0,   0),
    ),
    chess.QUEEN: (
        (-20, -10, -10,  -5,  -5, -10, -10, -20),
        (-10,   0,   0,   0,   0,   0,   0, -10),
        (-10,   0,   5,   5,   5,   5,   0, -10),
        ( -5,   0,   5,   5,   5,   5,   0,  -5),
        (  0,   0,   5,   5,   5,   5,   0,  -5),
        (-10,   5,   5,   5,   5,   5,   0, -10),
        (-10,   0,   5,   0,   0,   0,   0, -10),
        (-20, -10, -10,  -5,  -5, -10, -10, -20),
    ),
    chess.ROOK: (
        (  0,   0,   0,   0,   0,   0,   0,   0),
        (  5,  10,  10,  10,  10,  10,  10,   5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        ( -5,   0,   0,   0,   0,   0,   0,  -5),
        (  0,   0,   0,   5,   5,   0,   0,   0),
    ),
}

In der Endphase wird für den König eine andere Positionstabelle verwendet.
Diese ist nachfolgend definiert und wird der Variablen `kings_end_game_squared_table` zugewiesen.

In [ ]:
kings_end_game_squared_table = (
    (-50, -40, -30, -20, -20, -30, -40, -50),
    (-30, -20, -10,   0,   0, -10, -20, -30),
    (-30, -10,  20,  30,  30,  20, -10, -30),
    (-30, -10,  30,  40,  40,  30, -10, -30),
    (-30, -10,  30,  40,  40,  30, -10, -30),
    (-30, -10,  20,  30,  30,  20, -10, -30),
    (-30, -30,   0,   0,   0,   0, -30, -30),
    (-50, -30, -30, -30, -30, -30, -30, -50),
)

Das Paket Python-Chess weist jeder Positionen einen Ganzzahlwert zu.
So erhält das Feld A1 den Zahlenwert 0, das Feld B1 den Wert 1.
Diese Zuweisung lässt sich bis zum letzten Feld H8 fortführen, das den Zahlenwert 63 erhält.
Um nachfolgend effizient auf die Positionswerte basierend auf dem Zahlenwert des Feldes zugreifen zu können, müssen die Zeilen der Positionstabellen umgekehrt werden, sodass Zeile 1 den Index 0 und die Zeile 8 den Index 7 erhält.
Dafür wird im Folgenden über jedes Schlüssel-Wert-Paar des `piece_squared_tables` Dictionary iteriert, die Tabelle in eine Liste konvertiert, diese Liste dann umgekehrt und anschließend in ein Tupel zurück überführt.
Analog wird die eine Tabelle, die in der Variablen `kings_end_game_squared_table` abgelegt ist, in eine Liste überführt und deren Elemente in umgekehrter Reihenfolge als Tupel in die Variable zurückgeschrieben.

In [ ]:
piece_squared_tables = {key: tuple(reversed(list(value))) 
                        for key, value in piece_squared_tables.items()}
kings_end_game_squared_table = tuple(reversed(list(kings_end_game_squared_table)))

Die bisher definierten Positionstabellen sind aus der Sicht des Weißen bzw. maximierenden Spielers definiert.
Da der zu entwickelnde Schachcomputer aber beide Farben und damit auch gegen sich selber spielen können soll, müssen noch die Positionstabellen für den minimierenden Spieler generiert werden.
Hierzu werden wie beim maximierenden Spieler die Zeilen der Tabellen umgekehrt.
Weil sich aber die Tabelle als ganzen um 180° drehen muss, werden zusätzlich alle Spalten, was den Elementen innerhalb der Zeilen entspricht, umgedreht.
Das resultierende Dictionary bzw. die resultierende Positionstabelle für den König in der Endphase, werden in den zugehörigen Variablennamen mit dem Präfix `reversed_` abgelegt.

In [ ]:
reversed_piece_squared_tables = {key: tuple([
                                            piece[::-1]
                                            for piece in value][::-1]) 
                                 for key, value in piece_squared_tables.items()}
reversed_kings_end_game_squared_table = tuple([
                                        piece[::-1]
                                        for piece in kings_end_game_squared_table][::-1])

Die Funktion `get_piece_quared_tables_value` liefert den Wert der figurenspezifischen Positionstabellen für eine übergebene Figur `piece` auf dem Schachbrett.
Der Zeilen- und Spaltenindex in der Tabelle wird basierend auf dem Zahlenwert des Feldes, auf dem die Figur sich befindet (`square`), berechnet.
Dabei entspricht der Zeilenindex dem Ergebnis der Ganzzahldivision durch 8 und der Spaltenindex dem Rest aus der Division mit der Zahl 8.
Zudem wird ein Parameter `end_game` benötigt, der genau dann wahr ist, wenn sich das Spiel in der Endphase befindet und für den König eine abweichende Positionstabelle verwendet wird.
Des Weiteren wird erneut der gefundene Wert negiert, wenn es sich bei der Farbe um Schwarz und somit um den minimierenden Spieler handelt.

In [ ]:
def get_piece_squared_tables_value(piece: chess.Piece, square: int, end_game: bool=False) -> int:
    factor = -1 if piece.color == chess.BLACK else 1
    row = square // 8
    column = square % 8
    
    if end_game and piece.piece_type == chess.KING:
        if piece.color == chess.WHITE:
            return kings_end_game_squared_table[row][column]
        else:
            return reversed_kings_end_game_squared_table[row][column]
    
    if piece.color == chess.WHITE:
        piece_squared_table = piece_squared_tables.get(piece.piece_type)
    else:
        piece_squared_table = reversed_piece_squared_tables.get(piece.piece_type)
        
    return factor * piece_squared_table[row][column]

Die Funktion `simple_eval_heuristic` setzt die einfache Bewertungsheuristik um.
Sie erhält als Eingabeparameter die aktuelle Stellung `board` und ob sich das Spiel in der Endphase befindet (`end_game`).
Die Funktion schaut sich jedes Feld des Schachbretts an.
Befindet sich eine Figur auf dem Feld, so werden der Figurenwert und der Positionswert bestimmt und zum Endergebnis, das in der Variablen `piece_value` gespeichert wird, addiert.
Das Ergebnis wird dann zurückgegeben.

In [ ]:
def simple_eval_heuristic(board: chess.Board, end_game: bool = False) -> int:
    piece_value = 0
    for square in range(64):
        piece = board.piece_at(square)
        if not piece:
            continue
        piece_value += get_piece_value(piece)
        piece_value += get_piece_squared_tables_value(piece, square, end_game)
    return piece_value

Nachdem die verwendete Bewertungsheuristik implementiert ist, kann nun das Alpha-Beta Pruning zusammen mit dem Iterative Deepening umgesetzt werden.
Hierfür wird eine Funktion `iterative_deepening` implementiert, die den Ablauf dieses Prozesses koordiniert.
Sie erhält folgende drei Parameter:

- Der Parameter `board` enthält die aktuelle Stellung.
- Die initiale Tiefe, bei der gesucht wird, wird mit dem Parameter `depth` übergeben.
- `color` repräsentiert den Spieler, der aktuell am Zug ist.

Alle anderen benötigten Informationen werden global abgelegt.
Zu Beginn des Iterative Deepening wird geprüft, ob der Spieler nur einen erlaubten Zug machen kann.
Ist dies der Fall, so wird er sofort zurückgegeben und keine weiteren Berechnungen sind von Nöten.
Ist mehr als ein zulässiger Zug möglich, so wird die aktuelle Systemzeit global gespeichert und die Variable `d` mit 0 initialisiert.
`d` repräsentiert hierbei die Tiefe, die zur initialen Tiefe `depth` hinzugefügt wird.
In einer Endlosschleife wird sukzessiv die Suchtiefe erhöht und `minimize` bzw. `maximize` aufgerufen.
Sowohl `alpha` als auch `beta` werden mit einem sehr hohen bzw. sehr niedrigen Zahlenwert initialisiert.
Bei jedem Schleifendurchlauf wird dabei dem globale besten Zug (`global_best_move`) der Zug aus der Variablen `best_move` zugewiesen.
Des Weiteren wird überprüft, ob die Zeit des Schachcomputers abgelaufen ist.
Ist dies der Fall, wird der globale beste Zug zurückgegeben.

Um das starke Anwachsen des `move_scores` Dictionary zu verhindert, wird dieses am Ende des Zuges des Schachcomputers beschnitten.
Zum weiteren Verständnis dient folgende Grafik.

![neighbor-boards](neighbor-boards.png)

Die einzelnen Knoten des Graphen repräsentieren die möglichen Stellungen, wobei der Knoten 0 die aktuelle Stellung darstellt.
Entscheidet sich der Spieler für den linken Pfad (zum Knoten 1.1), so werden die Stellungen zu den Knoten 0 und 1.2 gelöscht.
Entscheidet sich im darauffolgenden Zug der aktuelle Spieler für den rechten Pfad (zum Knoten 2.1.2), so wird der Elternknoten 1.1 sowie alle Knoten auf der selben Ebene (2.1.1, 2.2.1 und 2.2.2) gelöscht.
Auf diese Weise wird das starke Anwachsen des `move_scores` Dictionary eingeschränkt.
Bei dieser Lösung handelt es sich um eine recht einfache Lösungen, die zwar optimiert werden kann, jedoch nicht weiter verfolgt wird.

In [ ]:
def iterative_deepening(board: chess.Board, depth: int, color: int):
    global best_move
    global current_depth
    global global_best_move
    global is_timeout
    global neighbor_boards
    global start_time

    if board.legal_moves == 1:
        return board.legal_moves[0]

    is_timeout = False
    start_time = time.time()
    d = 0

    while True:
        if d > 1:
            global_best_move = best_move
            print(
                f"Completed search with depth {current_depth}. Best move so far: {global_best_move} (Score: {move_scores[board.fen()][global_best_move]})"
            )
        current_depth = depth + d
        if color == chess.BLACK:
            minimize(board, current_depth, -sys.maxsize, sys.maxsize, color)
        else:
            maximize(board, current_depth, -sys.maxsize, sys.maxsize, color)
        d += 1
        if is_timeout:
            del move_scores[board.fen()]
            newboard = board.copy()
            newboard.push(global_best_move)
            for neighbor_board in neighbor_boards[1]:
                if neighbor_board != newboard.fen():
                    try:
                        del move_scores[neighbor_board]
                    except KeyError:
                        pass
            neighbor_boards = defaultdict(
                list,
                {key - 1: value for key, value in neighbor_boards.items() if key != 1},
            )
            return global_best_move

Die Implementierung der Funktion `maximize` orientiert sich stark am Pseudocode aus dem Theorieteil.
Die Funktion erhält fünf Parameter.

- `board` enthält die aktuelle Stellung.
- `depth` repräsentiert die Tiefe, mit der `maximize` aufgerufen wird.
- `alpha` repräsentiert die Variable Alpha des Alpha-Beta Prunings.
- `beta` repräsentiert die Variable Beta des Alpha-Beta Prunings.
- `color` steht für den Spieler, der gerade am Zug ist.

Zu Beginn wird geprüft, ob die Zeit des Schachcomputers bereits abgelaufen ist.
Ist dies der Fall, so wird die Variable `is_timeout` auf `True` gesetzt und `alpha` zurückgegeben.
Ist die Zeit noch nicht abgelaufen wird des Weiteren überprüft, ob es sich bei der aktuellen Suchtiefe um eine Tiefe kleiner als 1 handelt.
Ist dies der Fall, wird die aktuelle Stellung mittels der Bewertungsheuristik evaluiert und der entsprechende Wert zurückgeliefert.
Anschließend werden alle zulässigen Züge nach ihren Scores sortiert, sodass aussichtsreichere Züge zuerst evaluiert werden.
Ist kein Score für einen zulässigen Zug vorhanden, so wird diesem der für den Spieler schlechteste Score (`-sys.maxsize`) zugewiesen.
Anschließend wird über alle zulässigen Züge iteriert und die Funktion `minimize` nach der Ausführung des Zuges des aktuellen Schleifendurchlaufs aufgerufen.
Die Variablen `score` und `alpha` werden basierend auf dem Wert von `move_score` aktualisiert, wie dies im Pseudocode aus dem Theorieteil der Fall ist.
Zusätzlich wird `best_move` der aktuelle Zug zugewiesen, wenn folgende drei Bedingungen erfüllt sind:

1. Der aktuelle Score ist größer als Alpha, wodurch die Variable `alpha` aktualisiert wird und ein möglicher neuer bester Zug gefunden wurde.
1. Die aktuelle Suchtiefe entspricht der Suchtiefe, mit der die Funktion `iterative_deepening` die aktuelle Iteration anstieß.
1. Beim Spieler, der derzeit am Zug ist, handelt es sich um den Weißen bzw. maximierenden Spieler.

Während des Schleifendurchlaufs wird der `move_score` für einen Zug im Dictionary `move_scores` abgelegt.
Dabei wird er der Stellung zugewiesen, die als Ausgangsstellung für den Zug dient.

In [ ]:
def maximize(board: chess.Board, depth: int, alpha: int, beta: int, color: int) -> int:
    global best_move
    global global_best_move
    global is_timeout
    global neighbor_boards
    global start_time

    if time.time() - start_time > TIMEOUT_SECONDS:
        is_timeout = True
        return alpha

    if depth < 1:
        return simple_eval_heuristic(board)

    score = alpha

    board_scores = move_scores.get(board.fen(), dict())
    moves = sorted(
        board.legal_moves, key=lambda move: -board_scores.get(move, -sys.maxsize),
    )

    for move in moves:
        newboard = board.copy()
        newboard.push(move)
        move_score = minimize(newboard, depth - 1, alpha, beta, color)
        move_scores[board.fen()][move] = move_score
        neighbor_boards[current_depth - depth + 1].append(newboard.fen())
        newboard.pop()

        if move_score > score:
            score = move_score

            if score >= alpha:
                alpha = score
                if depth == current_depth and color == chess.WHITE:
                    best_move = move
        if alpha >= beta:
            return alpha
    return score

Die Funktion `minimize` ist zu großen Teilen mit der Funktion `maximize` identisch, wie dies auch im Pseudocode aus dem Theorieteil der Fall ist.
Die Unterschiede zur `maximize`-Implementierung sind:

- Die Liste der zulässigen Züge ist aufwärts und nicht abwärts sortiert.
Somit fängt der Schachcomputer mit dem Zug an, der den geringsten Score besitzt.
Zudem ist der für den Spieler schlechteste Zug nun `sys.maxsize`.
- Es wird die Variable `beta` aktualisiert und nicht die Variable `alpha`.
Diese wird zudem aktualisiert, wenn der aktuelle `score` niedriger und somit besser ist.
- Eine der Bedingungen für die Aktualisierung des `best_move` ändert sich: So wird nun die Variable nur gesetzt, wenn der Schwarze bzw. minimierende Spieler am Zug ist.

In [ ]:
def minimize(board: chess.Board, depth: int, alpha: int, beta: int, color: int) -> int:
    global best_move
    global global_best_move
    global is_timeout
    global neighbor_boards
    global start_time

    if time.time() - start_time > TIMEOUT_SECONDS:
        is_timeout = True
        return beta

    if depth < 1:
        return simple_eval_heuristic(board)

    score = beta

    board_scores = move_scores.get(board.fen(), dict())
    moves = sorted(
        board.legal_moves, key=lambda move: board_scores.get(move, sys.maxsize),
    )

    for move in moves:
        newboard = board.copy()
        newboard.push(move)
        move_score = maximize(newboard, depth - 1, alpha, beta, color)
        move_scores[board.fen()][move] = move_score
        neighbor_boards[current_depth - depth + 1].append(newboard.fen())
        newboard.pop()

        if move_score < score:
            score = move_score

            if score <= beta:
                beta = score
                if depth == current_depth and color == chess.BLACK:
                    best_move = move
        if beta <= alpha:
            return beta
    return score

Die Funktion `get_opening_data_base_moves` ist eine Hilfsfunktion, die einen Zug aus der Opening Data Base zurückgibt, falls ein Zug für die übergebene Stellung gefunden wird.

In [ ]:
def get_opening_data_base_moves(board: chess.Board):
    move = None
    opening_moves = []
    with chess.polyglot.open_reader("Performance.bin") as reader:
        for entry in reader.find_all(board):
            opening_moves.append(entry)
    if opening_moves:
        random_entry = random.choice(opening_moves)
        move = random_entry.move
        print(move)
    return move

Nachdem alle notwendigen Funktionen für den Schachcomputer implementiert sind, kann nun die Funktion `ai_player` implementiert werden.
Sie repräsentiert den Schachcomputer dem Spiel gegenüber.
Die Funktion erhält zwei Parameter:

- `board` enthält die aktuelle Stellung.
- `color` repräsentiert den Spieler, der am Zug ist.

Zuerst wird in der verfügbaren Opening Data Base geschaut, ob ein Zug für die übergebene Stellung vorhanden ist.
Ist dies der Fall, so wird der Zug zurückgegeben.
Andernfalls wird das globale `move_scores` Dictionary zurückgesetzt und die Funktion `iterative_deepening` aufgerufen und deren gewählter Zug zurückgegeben.

In [ ]:
def ai_player(board: chess.Board, color: int):
    move = get_opening_data_base_moves(board)
    
    if move:
        return move
    else:
        print(len(move_scores))
        print(len(neighbor_boards))
        return iterative_deepening(board, 0, color)

Nachdem der Schachcomputer implementiert ist, wird nun die Schnittstelle für den menschlichen Spieler implementiert.
Hierfür wird zuerst eine Hilfsfunktion `get_move` benötigt, die die Nutzereingabe in einen Zug umwandelt, sofern dies möglich ist.
Zudem wird in ihr überprüft, ob das Spiel vorzeitig zu beenden ist.
Dies ist der Fall, wenn der Nutzer als Zug ein `q` (für engl. *quit*) eingibt.
Sie erhält als Parameter den dem Nutzer anzuzeigenen Text für die Zugeingabe.

In [ ]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

Die Funktion `human_player` repräsentiert den menschlichen Spieler und koordiniert die Züge des Spielers.
Dafür wird zuerst die aktuelle Stellung mittels der IPython-Funktion `display` angezeigt.
Anschließend werden dem menschlichen Spieler alle zulässigen Züge für die aktuelle Stellung angezeigt und er wird aufgefordert, seinen Zug zu tätigen.
Diese Aufforderung geschieht solange, bis der Nutzer einen gültigen Zug eingegeben hat oder aber das Abbruchkriterium `q`.

In [ ]:
def human_player(board):
    display(board)
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    print(f"Legal moves: {(', '.join(sorted(legal_uci_moves)))}")
    uci = get_move(f"{who(board.turn)}'s move [q to quit]>")
    while uci not in legal_uci_moves:
        print(f"Legal moves: {(', '.join(sorted(legal_uci_moves)))}")
        uci = get_move(f"{who(board.turn)}'s move [q to quit]>")
    return uci

Für die Anzeige werden die Spielernamen benötigt.
Hierfür wird eine Hilfsfunktion `who` implementiert, die einen Spieler als Parameter erhält und den zugehörigen Namen als Zeichenkette zurückgibt.

In [ ]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

`play_game` ist die Funktion, die den Spielablauf koordiniert.
Der Funktion kann optional ein Parameter `pause` übergeben werden.
Dieser legt fest, wie lange die Stellung und der zu ihr geführte Zug angezeigt werde soll, bevor der andere Spieler am Zug ist.
Der übergebene Zahlenwert wird als Anzahl an Sekunden interpretiert.
Nach der Initialisierung des Schachbretts sind solange beide Spieler abwechseld am Zug, bis das Spiel als beendet angesehen wird oder das Spiel abgebrochen wurde.
Anschließend werden je nach Ausgang des Spiels unterschiedliche Ergebnisse angezeigt.

In [ ]:
def play_game(pause=1):
    board = chess.Board()
    
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                #move = board.parse_uci(human_player(board))
                move = ai_player(board, chess.WHITE)
            else:
                move = ai_player(board, chess.BLACK)
                
            name = who(board.turn)
            board.push(move)
            html = "<br/>%s </b>"%(board._repr_svg_())
            clear_output(wait=True)
            display(HTML(html))
            time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted"
        return None, msg, board
    
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: fivefold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    
    print(msg)
    return result, msg, board

In [ ]:
play_game()